In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns   
from pivottablejs import pivot_ui
import datetime
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

# from sklearn._ import
# from scipy.stats import 
# import statsmodels.api as sm


In [2]:
nlms = pd.read_csv('data/NLMS_PublicUse_5.1/11_new.csv', index_col='record');

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
nlms.columns

Index(['age', 'race', 'sex', 'ms', 'hisp', 'adjinc', 'educ', 'pob', 'wt',
       'hhid', 'hhnum', 'reltrf', 'occ', 'majocc', 'ind', 'majind', 'esr',
       'urban', 'smsast', 'inddea', 'cause113', 'follow', 'dayod', 'hosp',
       'hospd', 'ssnyn', 'vt', 'histatus', 'hitype', 'povpct', 'stater',
       'rcow', 'tenure', 'citizen', 'health', 'indalg', 'smok100', 'agesmk',
       'smokstat', 'smokhome', 'curruse', 'everuse'],
      dtype='object')

In [4]:
cols = ['age', 'race', 'sex', 'ms', 'hisp', 'adjinc', 'educ', 'pob',
       'wt', 'hhid', 'hhnum', 'reltrf', 'occ', 'majocc', 'ind', 'majind',
       'esr', 'urban', 'smsast', 'inddea', 'cause113', 'follow', 'dayod',
       'hosp', 'hospd', 'ssnyn', 'vt', 'histatus', 'hitype', 'povpct',
       'stater', 'rcow', 'tenure', 'citizen', 'health', 'indalg', 'smok100',
       'agesmk', 'smokstat', 'smokhome', 'curruse', 'everuse']

In [5]:
col_dict = {'index': 'record number', 'age': 'age at time of interview', 'race': 'race', 'sex': 'gender', 
            'ms': 'marital status', 'hisp': 'hispanic', 'adjinc': 'inflation adjusted income', 'educ': 'highest grade completed', 
            'pob': 'region of birth', 'wt': 'adjusted weight', 'hhid': 'household ID number', 'hhnum': 'number of people in household',
            'reltrf': 'relationship to reference person', 'occ': '4 digit occupation code', 'majocc': 'major occupation code',
            'ind': '4 digit industry code', 'majind': 'major industry code', 'esr': 'employment status recode', 
            'urban': 'urban / rural status', 'smsast': 'SMSAST (Standard Metropolitan Statistical Area) status', 'inddea': 'death indicator',
            'cause113': 'cause of death', 'follow': 'length of follow-up', 'dayod': 'day of week of death', 
            'hosp': 'hospital type', 'hosppd': 'hospital death indicator', 'ssnyn': 'presence of ssn', 
            'vt': 'veteran status', 'histatus': 'health insurance status', 'hitype': 'health insurance type', 
            'povpct': 'income as percent of poverty level', 'stater': 'state recode', 'rcow': 'recoded class of worker', 
            'tenure': 'housing tenure', 'citizen': 'citizenship', 'health': 'health', 'indalg': 'indicator of algorithmic death', 
            'smok100': 'smoked more than 100 cigarettes', 'agesmk': 'age started smoking',
            'smokstat': 'cigarette smoking status', 'smokhome': 'rules for smoking cigarettes in the home', 
            'curruse': 'currently use smokeless tobacco', 'everuse': 'ever use smokeless tobacco'}

In [6]:
print(col_dict)

{'index': 'record number', 'age': 'age at time of interview', 'race': 'race', 'sex': 'gender', 'ms': 'marital status', 'hisp': 'hispanic', 'adjinc': 'inflation adjusted income', 'educ': 'highest grade completed', 'pob': 'region of birth', 'wt': 'adjusted weight', 'hhid': 'household ID number', 'hhnum': 'number of people in household', 'reltrf': 'relationship to reference person', 'occ': '4 digit occupation code', 'majocc': 'major occupation code', 'ind': '4 digit industry code', 'majind': 'major industry code', 'esr': 'employment status recode', 'urban': 'urban / rural status', 'smsast': 'SMSAST (Standard Metropolitan Statistical Area) status', 'inddea': 'death indicator', 'cause113': 'cause of death', 'follow': 'length of follow-up', 'dayod': 'day of week of death', 'hosp': 'hospital type', 'hosppd': 'hospital death indicator', 'ssnyn': 'presence of ssn', 'vt': 'veteran status', 'histatus': 'health insurance status', 'hitype': 'health insurance type', 'povpct': 'income as percent of p

In [7]:
lens = nlms.apply(lambda x: len(x.unique()))
lens

age             91
race             6
sex              2
ms               6
hisp             4
adjinc          15
educ            15
pob             64
wt             930
hhid        745424
hhnum           21
reltrf           7
occ            365
majocc          12
ind            229
majind          15
esr              6
urban            3
smsast           4
inddea           2
cause113       106
follow        4019
dayod            8
hosp             7
hospd            6
ssnyn            2
vt               3
histatus         3
hitype           7
povpct          21
stater          51
rcow             6
tenure           4
citizen          6
health           6
indalg           3
smok100          1
agesmk           1
smokstat         1
smokhome         1
curruse          1
everuse          1
dtype: int64

In [8]:
nlms.pob.unique()

array([909,   0, 942, 928, 917, 925, 104, 111, 934, 945, 936, 944, 107,
       927, 926, 948, 906, 950, 937, 911, 933, 939, 106, 109, 912, 108,
       913, 102, 949, 103, 931, 954, 951, 924, 901, 918, 938, 956, 923,
       947, 921, 101, 105, 955, 941, 953, 905, 902, 920, 915, 904, 930,
       922, 940, 932, 908, 910, 946, 935, 110, 919, 916, 929, 960])

In [9]:
pob_dict = {909: 'CT',   0: np.nan, 942: 'PA', 928: 'MS', 917: 'IL', 925: 'MA', 104: 'Western Europe', 111: 'Other NS', 934: 'NJ', 945: 'SC', 936: 'NY', 944: 'RI', 107: 'Central America',
       927: 'MN', 926: 'MI', 948: 'TX', 906: 'CA', 950: 'VT', 937: 'NC', 911: 'DC', 933: 'NH', 939: 'OH', 106: 'South America', 109: 'Island Country', 912: 'FL', 108: 'Mexico',
       913: 'GA', 102: 'Asia', 949: 'UT', 103: 'Eastern Europe', 931: 'NE', 954: 'WV', 951: 'VA', 924: 'MD', 901: 'AL', 918: 'IN', 938: 'ND', 956: 'WY', 923: 'ME',
       947: 'TN', 921: 'KY', 101: 'Africa', 105: 'North Am, Not US or Mexico', 955: 'WI', 941: 'OR', 953: 'WA', 905: 'AR', 902: 'AK', 920: 'KS', 915: 'HI', 904: 'AZ', 930: 'MT',
       922: 'LA', 940: 'OK', 932: 'NV', 908: 'CO', 910: 'DE', 946: 'SD', 935: 'NM', 110: 'Other NS', 919: 'IA', 916: 'ID', 929: 'MO', 960: 'American Samoa'}

In [10]:
print('Pre na qty: ' + str(nlms.isnull().sum() * 100 / len(nlms)))

Pre na qty: age           0.000000
race          0.157487
sex           0.000000
ms           19.684568
hisp          2.861032
adjinc        2.412385
educ         19.120231
pob           0.000000
wt            0.000000
hhid          0.000000
hhnum         0.000000
reltrf        0.254704
occ          46.609888
majocc       46.609888
ind          46.621931
majind       46.621931
esr          19.122029
urban         0.710708
smsast        0.711471
inddea        0.000000
cause113      0.000000
follow        0.000000
dayod        91.240126
hosp         91.401209
hospd        92.088921
ssnyn         0.000000
vt           21.582314
histatus     31.484596
hitype       31.484596
povpct        0.000000
stater        0.000000
rcow         46.521880
tenure        1.386703
citizen      74.667969
health       79.067361
indalg       80.994806
smok100     100.000000
agesmk      100.000000
smokstat    100.000000
smokhome    100.000000
curruse     100.000000
everuse     100.000000
dtype: float64


In [11]:
nlms.follow.value_counts(normalize=True)

4018    0.912401
3617    0.000039
2531    0.000036
3199    0.000036
3565    0.000035
          ...   
7       0.000010
478     0.000009
3       0.000007
1       0.000007
0       0.000001
Name: follow, Length: 4019, dtype: float64

In [12]:
nlms.drop(columns=['smok100', 'agesmk', 'smokstat', 'smokhome', 'curruse', 'everuse'], inplace=True)

In [13]:
nlms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1835072 entries, 88426 to 670
Data columns (total 36 columns):
 #   Column    Dtype  
---  ------    -----  
 0   age       int64  
 1   race      float64
 2   sex       int64  
 3   ms        float64
 4   hisp      float64
 5   adjinc    float64
 6   educ      float64
 7   pob       int64  
 8   wt        int64  
 9   hhid      int64  
 10  hhnum     int64  
 11  reltrf    float64
 12  occ       float64
 13  majocc    float64
 14  ind       float64
 15  majind    float64
 16  esr       float64
 17  urban     float64
 18  smsast    float64
 19  inddea    int64  
 20  cause113  int64  
 21  follow    int64  
 22  dayod     float64
 23  hosp      float64
 24  hospd     float64
 25  ssnyn     int64  
 26  vt        float64
 27  histatus  float64
 28  hitype    float64
 29  povpct    int64  
 30  stater    int64  
 31  rcow      float64
 32  tenure    float64
 33  citizen   float64
 34  health    float64
 35  indalg    float64
dtypes: float

In [14]:
nlms.shape

(1835072, 36)

### DataFrame for deceased individuals

In [36]:
dec = nlms[nlms.dayod.isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0])]


In [37]:
dec.shape

(160750, 36)

In [38]:
dec.to_csv('data/dec.csv')

In [165]:
# Start over here: 
dec = pd.read_csv('data/dec.csv')

In [166]:
# Split before preprocessing
dec_label = dec.pop('age')
dec_features = dec.copy()

In [167]:
dec_feat_train, dec_feat_test, dec_label_train, dec_label_test = train_test_split(dec_features, dec_label, test_size=0.2)

#### Pipeline for Processing

In [168]:
cols_to_drop = ['hhid', 'reltrf', 'occ', 'majocc', 'majind', 'ind', 'smsast', 
                'inddea', 'follow', 'hospd', 'stater', 'health', 'indalg',
               'citizen', 'rcow']

In [169]:
dec_feat_train.drop(columns=cols_to_drop, inplace=True, axis=1);

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [170]:
dec_feat_train.isnull().sum() * 100 / len(dec_feat_train)

record       0.000000
race         0.136858
sex          0.000000
ms           1.013997
hisp         2.625194
adjinc       2.262830
educ         0.950233
pob          0.000000
wt           0.000000
hhnum        0.000000
esr          0.936236
urban        0.101866
cause113     0.000000
dayod        0.000000
hosp         1.815708
ssnyn        0.000000
vt           1.172628
histatus    30.730171
hitype      30.730171
povpct       0.000000
tenure       0.847589
dtype: float64

In [171]:
dec_feat_train.columns

Index(['record', 'race', 'sex', 'ms', 'hisp', 'adjinc', 'educ', 'pob', 'wt',
       'hhnum', 'esr', 'urban', 'cause113', 'dayod', 'hosp', 'ssnyn', 'vt',
       'histatus', 'hitype', 'povpct', 'tenure'],
      dtype='object')

In [172]:
cat_cols = ['race', 'sex', 'ms', 'hisp', 'pob', 'esr', 'urban', 
               'cause113', 'dayod', 'hosp', 'ssnyn', 'vt', 'histatus', 'hitype', 'tenure']

num_cols = ['adjinc', 'educ', 'wt', 'hhnum']

Fill missing values

In [173]:
imputer = KNNImputer(n_neighbors=1)

In [175]:
dec_feat_train.set_index('record', inplace=True)

In [179]:
cat_df_imp = pd.DataFrame(imputer.fit_transform(dec_feat_train[cat_cols]), columns=cat_cols, index=dec_feat_train.index)

In [180]:
cat_df_imp

,race,sex,ms,hisp,pob,esr,urban,cause113,dayod,hosp,ssnyn,vt,histatus,hitype,tenure
record,,,,,,,,,,,,,,,
970788,1.0,2.0,1.0,3.0,937.0,5.0,1.0,27.0,2.0,4.0,1.0,0.0,1.0,1.0,1.0
842203,1.0,1.0,1.0,3.0,918.0,4.0,1.0,52.0,6.0,1.0,1.0,0.0,1.0,1.0,1.0
835196,2.0,2.0,2.0,3.0,901.0,1.0,2.0,21.0,7.0,1.0,1.0,0.0,1.0,1.0,3.0
522533,1.0,2.0,3.0,3.0,936.0,1.0,1.0,55.0,6.0,1.0,1.0,0.0,1.0,1.0,2.0
865793,1.0,1.0,1.0,3.0,936.0,5.0,1.0,67.0,2.0,2.0,1.0,0.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535321,1.0,2.0,2.0,3.0,936.0,5.0,1.0,55.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0
1446125,1.0,1.0,1.0,3.0,942.0,5.0,1.0,32.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
553998,1.0,1.0,1.0,3.0,926.0,4.0,1.0,73.0,4.0,4.0,0.0,1.0,1.0,1.0,1.0


In [181]:
imputer_num = KNNImputer(n_neighbors=3)

In [182]:
num_df_imp = pd.DataFrame(imputer_num.fit_transform(dec_feat_train[num_cols]), columns=num_cols, index=dec_feat_train.index)

In [186]:
df = cat_df_imp.merge(num_df_imp, left_index=True, right_index=True)


In [189]:
df

,race,sex,ms,hisp,pob,esr,urban,cause113,dayod,hosp,ssnyn,vt,histatus,hitype,tenure,adjinc,educ,wt,hhnum
record,,,,,,,,,,,,,,,,,,,
970788,1.0,2.0,1.0,3.0,937.0,5.0,1.0,27.0,2.0,4.0,1.0,0.0,1.0,1.0,1.0,14.000000,8.000000,71.0,2.0
842203,1.0,1.0,1.0,3.0,918.0,4.0,1.0,52.0,6.0,1.0,1.0,0.0,1.0,1.0,1.0,7.000000,8.000000,144.0,2.0
835196,2.0,2.0,2.0,3.0,901.0,1.0,2.0,21.0,7.0,1.0,1.0,0.0,1.0,1.0,3.0,4.000000,10.000000,112.0,2.0
522533,1.0,2.0,3.0,3.0,936.0,1.0,1.0,55.0,6.0,1.0,1.0,0.0,1.0,1.0,2.0,2.000000,8.000000,178.0,1.0
865793,1.0,1.0,1.0,3.0,936.0,5.0,1.0,67.0,2.0,2.0,1.0,0.0,1.0,1.0,2.0,4.000000,5.000000,197.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535321,1.0,2.0,2.0,3.0,936.0,5.0,1.0,55.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0,12.000000,4.000000,123.0,6.0
1446125,1.0,1.0,1.0,3.0,942.0,5.0,1.0,32.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.000000,12.000000,56.0,2.0
553998,1.0,1.0,1.0,3.0,926.0,4.0,1.0,73.0,4.0,4.0,0.0,1.0,1.0,1.0,1.0,8.666667,4.000000,228.0,5.0


In [191]:
dec_feat_train = df.copy()

In [192]:
saved = dec_feat_train.copy()

In [193]:
saved.to_csv('data/saved.csv')

### For tomorrow: How to OHE and keep df in order

In [ ]:
dec_ft_enc = pd.get_dummies(dec_feat_train, prefix_sep="__",
                              columns=cat_cols, drop_first=True)

In [127]:
df = pd.get_dummies(dec_feat_train, columns = cat_cols, prefix=cat_cols, prefix_sep='__', drop_first=True).reset_index(drop=True)

In [130]:
df.set_index('record', inplace=True)

### One-Hot Encoding for Categorical Variables

In [30]:
dec_feat_train.columns

Index(['race', 'sex', 'ms', 'hisp', 'adjinc', 'educ', 'pob', 'wt', 'hhnum',
       'esr', 'urban', 'cause113', 'dayod', 'hosp', 'ssnyn', 'vt', 'histatus',
       'hitype', 'povpct', 'tenure'],
      dtype='object')

In [32]:
dec_ft_enc = pd.get_dummies(dec_feat_train, prefix_sep="__",
                              columns=cat_cols, drop_first=True)

In [288]:
dec_ = pd.concat([dec.drop('race', axis=1), pd.get_dummies(dec['race'], prefix='race', drop_first=True)], axis=1)

In [289]:
dec_2 = pd.concat([dec_.drop('sex', axis=1), pd.get_dummies(dec['sex'], prefix='gender', drop_first=True)], axis=1)

In [290]:
dec_3 = pd.concat([dec_2.drop('ms', axis=1), pd.get_dummies(dec_2['ms'], prefix='ms', drop_first=True)], axis=1)

In [291]:
dec_4 = pd.concat([dec_3.drop('hisp', axis=1), pd.get_dummies(dec_3['hisp'], prefix='hisp', drop_first=True)], axis=1)

In [292]:
dec_5 = pd.concat([dec_4.drop('pob', axis=1), pd.get_dummies(dec_4['pob'], prefix='pob', drop_first=True)], axis=1)

In [293]:
dec_5.drop(columns=['hhid', 'reltrf'], axis=1, inplace=True)

In [294]:
dec_6 = pd.concat([dec_5.drop('vt', axis=1), pd.get_dummies(dec_4['vt'], prefix='vt', drop_first=True)], axis=1)

In [295]:
dec_7 = pd.concat([dec_6.drop('citizen', axis=1), pd.get_dummies(dec_4['citizen'], prefix='citizen', drop_first=True)], axis=1)

In [296]:
dec_7.drop(columns=['health'], axis=1, inplace=True)

In [297]:
dec_8 = pd.concat([dec_7.drop('esr', axis=1), pd.get_dummies(dec_7['esr'], prefix='esr', drop_first=True)], axis=1)

In [298]:
dec_8.drop(columns=['ind'], axis=1, inplace=True)

In [299]:
dec_9 = pd.concat([dec_8.drop('majind', axis=1), pd.get_dummies(dec_7['majind'], prefix='majind', drop_first=True)], axis=1)

In [300]:
dec_9.drop(columns=['occ'], axis=1, inplace=True)

In [301]:
dec_9.drop(columns=['majocc'], axis=1, inplace=True)

In [302]:
dec_10 = pd.concat([dec_9.drop('rcow', axis=1), pd.get_dummies(dec_9['rcow'], prefix='rcow', drop_first=True)], axis=1)

In [303]:
dec_11 = pd.concat([dec_10.drop('histatus', axis=1), 
                    pd.get_dummies(dec_10['histatus'], 
                                   prefix='histatus', drop_first=True)], axis=1)

In [305]:
dec_12 = pd.concat([dec_11.drop('hitype', axis=1), 
                    pd.get_dummies(dec_11['hitype'], 
                                   prefix='hitype', drop_first=True)], axis=1)

In [307]:
dec_12.drop(columns='inddea', axis=1, inplace=True)

In [309]:
dec_13 = pd.concat([dec_12.drop('cause113', axis=1), 
                    pd.get_dummies(dec_12['cause113'], 
                                   prefix='cause113', drop_first=True)], axis=1)

In [311]:
dec_14 = pd.concat([dec_13.drop('dayod', axis=1), 
                    pd.get_dummies(dec_13['dayod'], 
                                   prefix='dayod', drop_first=True)], axis=1)

In [313]:
dec_15 = pd.concat([dec_14.drop('hosp', axis=1), 
                    pd.get_dummies(dec_14['hosp'], 
                                   prefix='hosp', drop_first=True)], axis=1)

In [315]:
dec_15.drop(columns='hospd', axis=1, inplace=True)

In [317]:
dec_15.drop(columns='indalg', axis=1, inplace=True)

In [319]:
dec_16 = pd.concat([dec_15.drop('tenure', axis=1), 
                    pd.get_dummies(dec_15['tenure'], 
                                   prefix='tenure', drop_first=True)], axis=1)

In [325]:
dec_17 = pd.concat([dec_16.drop('urban', axis=1), 
                    pd.get_dummies(dec_16['urban'], 
                                   prefix='urban', drop_first=True)], axis=1)

In [327]:
dec_17.drop(columns='smsast', axis=1, inplace=True)

In [329]:
dec_17.ssnyn.value_counts(normalize=True)

1    0.79768
0    0.20232
Name: ssnyn, dtype: float64

In [332]:
dec_17.drop(columns=['citizen_2.0', 'citizen_3.0', 'citizen_4.0', 'citizen_5.0'], axis=1, inplace=True)

In [334]:
dec_18 = pd.concat([dec_17.drop('ssnyn', axis=1), 
                    pd.get_dummies(dec_17['ssnyn'], 
                                   prefix='ssnyn', drop_first=True)], axis=1)

In [336]:
df_enc = dec_18.copy()

In [352]:
df_enc.drop(columns=['stater', 'days_to_fu', 'start_date'], inplace=True)

In [353]:
df_enc.to_csv('data/df_enc.csv')

In [361]:
df_enc.drop(columns=['dec_date', 'follow'], inplace=True, axis=1)

In [363]:
df_label = df_enc.pop("age")
df_features = df_enc.copy()

#### Splitting into Training & Testing Sets

In [367]:
df_feat_train, df_feat_test, df_label_train, df_label_test = train_test_split(df_label, df_features, random_state=42, test_size=0.20)

#### Filling nans with KNN imputer

In [354]:
df_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160750 entries, 88427 to 670
Columns: 230 entries, age to ssnyn_1
dtypes: float64(2), int64(5), object(1), uint8(222)
memory usage: 45.1+ MB


In [359]:
# scaler = MinMaxScaler()
# df = pd.DataFrame(scaler.fit_transform(df_enc), columns=df_enc.columns)
# df.head()

In [ ]:
imputer = KNNImputer(n_neighbors=5)
df2 = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)